In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
g

   user_id  prod_id        date  \
0     5044        0  2014-11-16   
1     5045        0  2014-09-08   
2     5046        0  2013-10-06   
3     5047        0  2014-11-30   
4     5048        0  2014-08-28   

                                              review  rating  label  
0  Drinks were bad, the hot chocolate was watered...     1.0     -1  
1  This was the worst experience I've ever had a ...     1.0     -1  
2  This is located on the site of the old Spruce ...     3.0     -1  
3  I enjoyed coffee and breakfast twice at Toast ...     5.0     -1  
4  I love Toast! The food choices are fantastic -...     5.0     -1  
length of data: 608598
check the null data: False


In [3]:
review=review3[['review']]
print(review.head())

                                              review
0  Drinks were bad, the hot chocolate was watered...
1  This was the worst experience I've ever had a ...
2  This is located on the site of the old Spruce ...
3  I enjoyed coffee and breakfast twice at Toast ...
4  I love Toast! The food choices are fantastic -...


In [5]:
!pip install spacy

  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached pathy-0.10.1-py3-none-any.whl (48 kB)
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached confection-0.0.4-py3-none-any.whl (32 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4
  Attempting

In [5]:
import spacy

In [14]:
!python -m spacy download en

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
sentence = "very rude"

In [8]:
doc=nlp(sentence)

In [184]:
length=len(review)
df=review.copy()
df['num_clauses']=np.nan

In [209]:
def check_clauses(doc):
    print(doc)
    if len(doc)!=0 and doc!=None:
        def find_root_of_sentence(doc):
            root_token = None
            for token in doc:
                if (token.dep_ == "ROOT"):
                    root_token = token
            return root_token
        root_token = find_root_of_sentence(doc)
        print(root_token)
        def find_other_verbs(doc, root_token):
            other_verbs = []
            for token in doc:
                ancestors = list(token.ancestors)
                if (token.pos_ == "VERB" and len(ancestors) == 1\
                    and ancestors[0] == root_token):
                    other_verbs.append(token)
            return other_verbs
        other_verbs = find_other_verbs(doc, root_token)
        print(other_verbs)
        def get_clause_token_span_for_verb(verb, doc, all_verbs):
            if len(verb)!=0:
                first_token_index = len(doc)
                last_token_index = 0
                this_verb_children = list(verb.children)
                #
                for child in this_verb_children:
                    if (child not in all_verbs):
                        if (child.i < first_token_index):
                            first_token_index = child.i
                        if (child.i > last_token_index):
                            last_token_index = child.i
                return(first_token_index, last_token_index)
        
        token_spans = []   
        all_verbs = [root_token] + other_verbs
        for other_verb in all_verbs:
            (first_token_index, last_token_index) = \
             get_clause_token_span_for_verb(other_verb, 
                                            doc, all_verbs)
            token_spans.append((first_token_index, 
                                last_token_index))
        sentence_clauses = []
        for token_span in token_spans:
            start = token_span[0]
            end = token_span[1]
            if (start < end):
                clause = doc[start:end]
                sentence_clauses.append(clause)
        sentence_clauses = sorted(sentence_clauses, 
                                  key=lambda tup: tup[0])
        clauses_text = [clause.text for clause in sentence_clauses]
        #print(clauses_text)
        return len(clauses_text)
    else:
        return 0

In [186]:
tt=check_clauses(doc)
print(tt)

[]
0


In [9]:
length=len(review)
df=review.copy()
df['num_clauses']=np.nan
print(df.head())

                                              review  num_clauses
0  Drinks were bad, the hot chocolate was watered...          NaN
1  This was the worst experience I've ever had a ...          NaN
2  This is located on the site of the old Spruce ...          NaN
3  I enjoyed coffee and breakfast twice at Toast ...          NaN
4  I love Toast! The food choices are fantastic -...          NaN


### Test with first 5 samples

In [194]:
for i in range(5):
    text=review.iloc[i,0].lower()
    text = text.replace("_", " ")
    text = text.replace("/", " ")
    text = text.replace("\\", " ")
    #text = text.replace("\\x", " ")   # deal with \x..........
    text = re.sub(br'(\xc2)(.)', b'', text.encode('utf-8')).decode()
    text = text.replace("can't", "can not")
    text = text.replace("won't", "will not")
    text = text.replace("'ve"," have")
    text = text.replace("'d"," had")
    text = text.replace("'m", " am")
    text = text.replace("'ll", " will")
    text = text.replace("'s", " is")
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("st.", "street")
    text = text.replace("bldg.", "building") 
    text = re.sub(r"\d+", "", text)
    text=re.split('[,.?!]',text)
    count=0
    for phrase in text:
        print(phrase)
        t=nlp(phrase)
        count+=check_clauses(t)
        print(count)
    df.iloc[i,1]=count
print(df.head())
    

drinks were bad
were
1
 the hot chocolate was watered down and the latte had a burnt taste to it
watered
3
 the food was also poor quality
was
4
 but the service was the worst part
was
5
 their cashier was very rude
was
6

6
this was the worst experience i have ever had a casual coffee light fare place
was
1
 the server disappeared for  minutes
disappeared
2
 just talking to his friend by the window as my girlfriend and i sat dumbfounded that this dude had the nerve to do that on the job
sat
3
 we are trying to make eye contact
trying
5
 but clearly getting paid to talk to his bud was more important to him
was
6
 my girlfriend went up to the counter once the server disappeared into the back for another  minutes (what is this guy doing
went
9
) and asked if she should order food up there or something
asked
10
 the girl at the counter gives her a weird look and just says "i will get your server
gives
12
" when they arrive from the back
arrive
13
 they look over at our table and have a la

In [12]:
def check_clauses(doc):
    #print('doc',doc)
    if len(doc)!=0 and doc!=None:
        def find_root_of_sentence(doc):
            root_token = None
            for token in doc:
                if (token.dep_ == "ROOT"):
                    root_token = token
            return root_token
        root_token = find_root_of_sentence(doc)
        if root_token!=None:
            #print('root token', root_token)
            def find_other_verbs(doc, root_token):
                other_verbs = []
                for token in doc:
                    ancestors = list(token.ancestors)
                    if (token.pos_ == "VERB" and len(ancestors) == 1\
                        and ancestors[0] == root_token):
                        other_verbs.append(token)
                return other_verbs
            other_verbs = find_other_verbs(doc, root_token)
            #print('other verbs',other_verbs)
            def get_clause_token_span_for_verb(verb, doc, all_verbs):
                if len(verb)!=0:
                    first_token_index = len(doc)
                    last_token_index = 0
                    this_verb_children = list(verb.children)
                    #
                    for child in this_verb_children:
                        if (child not in all_verbs):
                            if (child.i < first_token_index):
                                first_token_index = child.i
                            if (child.i > last_token_index):
                                last_token_index = child.i
                    return(first_token_index, last_token_index)

            token_spans = []   
            all_verbs = [root_token] + other_verbs
            for other_verb in all_verbs:
                (first_token_index, last_token_index) = \
                 get_clause_token_span_for_verb(other_verb, 
                                                doc, all_verbs)
                token_spans.append((first_token_index, 
                                    last_token_index))
            sentence_clauses = []
            for token_span in token_spans:
                start = token_span[0]
                end = token_span[1]
                if (start < end):
                    clause = doc[start:end]
                    sentence_clauses.append(clause)
            sentence_clauses = sorted(sentence_clauses, 
                                      key=lambda tup: tup[0])
            clauses_text = [clause.text for clause in sentence_clauses]
            #print(clauses_text)
            return len(clauses_text)
        else:
            return 0
    else:
        return 0

### Implement to whole dataset

In [13]:
length=len(review)
df=review.copy()
df['num_clauses']=np.nan
print(df.head())

                                              review  num_clauses
0  Drinks were bad, the hot chocolate was watered...          NaN
1  This was the worst experience I've ever had a ...          NaN
2  This is located on the site of the old Spruce ...          NaN
3  I enjoyed coffee and breakfast twice at Toast ...          NaN
4  I love Toast! The food choices are fantastic -...          NaN


In [14]:
%%time
for i in range(length):
    if i%1000==0:
        print(i)
    text=review.iloc[i,0].lower()
    text = text.replace("_", " ")
    text = text.replace("/", " ")
    text = text.replace("\\", " ")
    #text = text.replace("\\x", " ")   # deal with \x..........
    text = re.sub(br'(\xc2)(.)', b'', text.encode('utf-8')).decode()
    text = text.replace("can't", "can not")
    text = text.replace("won't", "will not")
    text = text.replace("'ve"," have")
    text = text.replace("'d"," had")
    text = text.replace("'m", " am")
    text = text.replace("'ll", " will")
    text = text.replace("'s", " is")
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("st.", "street")
    text = text.replace("bldg.", "building") 
    text = re.sub(r"\d+", "", text)
    text=re.split('[,.?!]',text)
    count=0
    for phrase in text:
        t=nlp(phrase)
        count+=check_clauses(t)
    df.iloc[i,1]=count

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [15]:
print(df.head())

                                              review  num_clauses
0  Drinks were bad, the hot chocolate was watered...          6.0
1  This was the worst experience I've ever had a ...         29.0
2  This is located on the site of the old Spruce ...          6.0
3  I enjoyed coffee and breakfast twice at Toast ...         27.0
4  I love Toast! The food choices are fantastic -...         16.0


In [16]:
from numpy import savetxt
savetxt("feature_num_clauses.txt",df,fmt="%s",delimiter="\t",encoding="utf-8")